In [1]:
import torch
#import numpy as np
#from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm
import zipfile
import pandas as pd
from sklearn.cluster import KMeans
from torch.utils.data.dataset import Dataset
import numpy as np
from torch.utils.data import DataLoader

In [5]:
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [6]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [8]:
n_users = len(ratings_df.userId.unique())
n_items = len(movies_df.movieId.unique())
print('unique user - ', n_users) 
print('unique movies - ', n_items) 
print('matrix size - ', n_items*n_users)
print('total ratings available - ', len(ratings_df))
print('% of filled matrix - ', len(ratings_df)/(n_users*n_items)*100,'%')

unique user -  610
unique movies -  9742
matrix size -  5942620
total ratings available -  100836
% of filled matrix -  1.6968273253211548 %


In [9]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=30):
        super().__init__()
        
        self.user_factors = torch.nn.Embedding(n_users, n_factors)  
        self.item_factors = torch.nn.Embedding(n_items, n_factors)  
        
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        users, items = data[:, 0], data[:, 1]
        return (self.user_factors(users) * self.item_factors(items)).sum(1)


    def predict(self, user, item):
        return self.forward(user, item)


In [10]:
rating_table = None

class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        
        self.userid2idx = {o:i for i, o in enumerate(users)}
        self.movieid2idx = {o:i for i, o in enumerate(movies)}
   

        
        self.idx2userid = {i:o for o, i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o, i in self.movieid2idx.items()}
        
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        global rating_table
        rating_table = self.ratings
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) 
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)


In [11]:
num_epochs = 5
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        
if cuda:
    model = model.cuda()

loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_set = Loader()
train_loader = DataLoader(train_set, 4096, shuffle=True)


Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9742, 8)
)
user_factors.weight tensor([[0.0311, 0.0374, 0.0240,  ..., 0.0384, 0.0279, 0.0189],
        [0.0337, 0.0431, 0.0290,  ..., 0.0158, 0.0105, 0.0365],
        [0.0316, 0.0475, 0.0238,  ..., 0.0074, 0.0464, 0.0212],
        ...,
        [0.0002, 0.0418, 0.0075,  ..., 0.0418, 0.0064, 0.0363],
        [0.0320, 0.0360, 0.0366,  ..., 0.0242, 0.0371, 0.0386],
        [0.0382, 0.0496, 0.0034,  ..., 0.0409, 0.0073, 0.0076]])
item_factors.weight tensor([[0.0027, 0.0328, 0.0287,  ..., 0.0316, 0.0058, 0.0439],
        [0.0200, 0.0409, 0.0310,  ..., 0.0277, 0.0380, 0.0045],
        [0.0384, 0.0006, 0.0096,  ..., 0.0023, 0.0163, 0.0201],
        ...,
        [0.0120, 0.0124, 0.0009,  ..., 0.0249, 0.0163, 0.0099],
        [0.0296, 0.0044, 0.0035,  ..., 0.0029, 0.0010, 0.0315],
        [0.0168, 0.0123, 0.0261,  ..., 0.0212, 0.0303, 0.0188]])


In [12]:
for it in tqdm(range(num_epochs)):
    losses= []
    for x,y in train_loader:
        if cuda:
            x,y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(),y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print('iter #{}'.format(it),'loss',sum(losses)/len(losses))
    

C:\Users\Dhiyanesh\AppData\Local\Temp\ipykernel_22316\3375793176.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/5 [00:00<?, ?it/s]

iter #0 loss 13.277543869018555
iter #1 loss 13.16233257293701
iter #2 loss 12.963232460021972
iter #3 loss 12.671710662841797
iter #4 loss 12.273095703125


In [13]:
c = 0
uw = 0
iw = 0

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name,param.data)
        if c == 0:
            uw = param.data
            c += 1
        else:
            iw = param.data


user_factors.weight tensor([[0.1822, 0.1889, 0.1772,  ..., 0.1913, 0.1804, 0.1700],
        [0.1538, 0.1651, 0.1474,  ..., 0.1316, 0.1290, 0.1567],
        [0.1384, 0.1537, 0.1292,  ..., 0.1147, 0.1504, 0.1232],
        ...,
        [0.1574, 0.1990, 0.1645,  ..., 0.1988, 0.1640, 0.1931],
        [0.1503, 0.1528, 0.1567,  ..., 0.1418, 0.1549, 0.1593],
        [0.1929, 0.2052, 0.1593,  ..., 0.1960, 0.1622, 0.1622]],
       device='cuda:0')
item_factors.weight tensor([[0.1565, 0.1856, 0.1826,  ..., 0.1851, 0.1589, 0.1961],
        [0.1468, 0.1680, 0.1572,  ..., 0.1567, 0.1646, 0.1308],
        [0.1850, 0.1470, 0.1555,  ..., 0.1489, 0.1622, 0.1657],
        ...,
        [0.0120, 0.0124, 0.0009,  ..., 0.0249, 0.0163, 0.0099],
        [0.0296, 0.0044, 0.0035,  ..., 0.0029, 0.0010, 0.0315],
        [0.0168, 0.0123, 0.0261,  ..., 0.0212, 0.0303, 0.0188]],
       device='cuda:0')


In [14]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()
print(trained_movie_embeddings.shape)

trained_user_embeddings = model.user_factors.weight.data.cpu().numpy()
print(trained_user_embeddings.shape)

(9742, 8)
(610, 8)


In [15]:
trained_user_embeddings = model.user_factors.weight.data.cpu().numpy()
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

print(f"Trained user embeddings shape: {trained_user_embeddings.shape}")
print(f"Trained movie embeddings shape: {trained_movie_embeddings.shape}")

full_matrix = np.dot(trained_user_embeddings, trained_movie_embeddings.T)
full_matrix.shape

Trained user embeddings shape: (610, 8)
Trained movie embeddings shape: (9742, 8)


(610, 9742)

In [16]:
# Assuming user index mapping (userId == 2 corresponds to index 1, since the first user is at index 0)
user_index = train_set.userid2idx[2]  # Index for user with userId 2
print(user_index)
# Generate predictions for all movies for this user
user_tensor = torch.tensor([user_index] * n_items)  # Repeat user index for all movies
movie_tensor = torch.tensor(list(range(n_items)))   # Movie indices from 0 to n_items-1

if cuda:
    user_tensor = user_tensor.cuda()
    movie_tensor = movie_tensor.cuda()

with torch.no_grad():
    predictions = model.forward(torch.stack([user_tensor, movie_tensor], dim=1)).cpu().numpy()

# Get the top 3 movie indices with highest predicted ratings
top_movie_indices = predictions.argsort()[-3:][::-1]

# Map back to original movie IDs and titles
top_movie_ids = [train_set.idx2movieid[idx] for idx in top_movie_indices]
top_movie_titles = movies_df[movies_df['movieId'].isin(top_movie_ids)][['movieId', 'title']]

print("Top 3 recommended movies for user with userId 2:")
print(top_movie_titles)


1
Top 3 recommended movies for user with userId 2:
      movieId                                              title
322       364                              Lion King, The (1994)
507       589                  Terminator 2: Judgment Day (1991)
3622     4973  Amelie (Fabuleux destin d'Amélie Poulain, Le) ...


In [24]:
len(user_tensor)
user_tensor

tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0')

In [20]:
len(movie_tensor)

9742

In [26]:
movie_tensor

tensor([   0,    1,    2,  ..., 9739, 9740, 9741], device='cuda:0')

In [27]:
predictions.shape

(9742,)

In [28]:
predictions

array([0.20664532, 0.17659886, 0.19018686, ..., 0.01328194, 0.01368707,
       0.02787149], dtype=float32)

In [29]:
full_matrix

array([[0.2605327 , 0.22355098, 0.24018405, ..., 0.01726888, 0.01628447,
        0.03593466],
       [0.20664532, 0.17659888, 0.19018687, ..., 0.01328194, 0.01368707,
        0.02787149],
       [0.19046508, 0.16465752, 0.17648977, ..., 0.01235066, 0.01186256,
        0.02644246],
       ...,
       [0.25475577, 0.21776052, 0.23324312, ..., 0.01682712, 0.01589837,
        0.03485052],
       [0.21431091, 0.18319498, 0.19723317, ..., 0.01402188, 0.01390625,
        0.02915715],
       [0.26129478, 0.2241462 , 0.24103537, ..., 0.01746325, 0.01650159,
        0.03581998]], dtype=float32)

In [17]:
# Step 1: Get predicted ratings for the user at index 2
user_index = 2
user_ratings = full_matrix[user_index]

# Step 2: Get the indices of the top 3 ratings
top_movie_indices = user_ratings.argsort()[-3:][::-1]  # Sort and get top 3 indices

# Step 3: Map indices back to original movie IDs and titles
top_movie_ids = [train_set.idx2movieid[idx] for idx in top_movie_indices]
top_movie_titles = movies_df[movies_df['movieId'].isin(top_movie_ids)][['movieId', 'title']]

print("Top 3 recommended movies for the user at index 2:")
print(top_movie_titles)


Top 3 recommended movies for the user at index 2:
      movieId                                              title
322       364                              Lion King, The (1994)
507       589                  Terminator 2: Judgment Day (1991)
3622     4973  Amelie (Fabuleux destin d'Amélie Poulain, Le) ...


In [4]:
import torch
import zipfile
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from torch.utils.data import Dataset, DataLoader

# Data Extraction
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# Dataset Loader Class
class Loader(Dataset):
    def __init__(self):
        ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')
        self.ratings = ratings_df.copy()
        
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        self.userid2idx = {o: i for i, o in enumerate(users)}
        self.movieid2idx = {o: i for i, o in enumerate(movies)}
        
        self.idx2userid = {i: o for o, i in self.userid2idx.items()}
        self.idx2movieid = {i: o for o, i in self.movieid2idx.items()}
        
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        self.x = torch.tensor(self.ratings.drop(['rating', 'timestamp'], axis=1).values)
        self.y = torch.tensor(self.ratings['rating'].values)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return len(self.ratings)

# Matrix Factorization Model
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=250):
        super(MatrixFactorization, self).__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        users, items = data[:, 0], data[:, 1]
        return (self.user_factors(users) * self.item_factors(items)).sum(1)
    
    def predict(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

# Main Recommender System Class
class RecommenderSystem:
    def __init__(self, n_factors=20, batch_size=8192, lr=1e-2, epochs=200):
        # Load data
        self.movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
        self.ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')
        
        # Initialize parameters
        self.n_users = len(self.ratings_df.userId.unique())
        self.n_items = len(self.movies_df.movieId.unique())
        self.model = MatrixFactorization(self.n_users, self.n_items, n_factors)
        self.loss_fn = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.epochs = epochs
        self.batch_size = batch_size
        self.cuda = torch.cuda.is_available()
        
        # Prepare DataLoader
        self.train_set = Loader()
        self.train_loader = DataLoader(self.train_set, batch_size=batch_size, shuffle=True)
        
        # Move model to GPU if available
        if self.cuda:
            self.model = self.model.cuda()
            
    def train(self):
        print("Training started...")
        for epoch in tqdm(range(self.epochs), desc="Epochs"):
            losses = []
            for x, y in self.train_loader:
                if self.cuda:
                    x, y = x.cuda(), y.cuda()
                
                self.optimizer.zero_grad()
                outputs = self.model(x)
                loss = self.loss_fn(outputs.squeeze(), y.type(torch.float32))
                losses.append(loss.item())
                loss.backward()
                self.optimizer.step()
                
            avg_loss = sum(losses) / len(losses)
            tqdm.write(f'Epoch {epoch+1}/{self.epochs}, Loss: {avg_loss:.4f}')
        print("Training completed.")
    
    def get_embeddings(self):
        user_embeddings = self.model.user_factors.weight.data.cpu().numpy()
        item_embeddings = self.model.item_factors.weight.data.cpu().numpy()
        return user_embeddings, item_embeddings
    
    def get_full_matrix(self):
        user_embeddings, item_embeddings = self.get_embeddings()
        return np.dot(user_embeddings, item_embeddings.T)
    
    def evaluate_model(self):
        print(f"Total Users: {self.n_users}")
        print(f"Total Movies: {self.n_items}")
        print(f"Matrix Size: {self.n_users * self.n_items}")
        print(f"Total Ratings Available: {len(self.ratings_df)}")
        print(f"% of Filled Matrix: {(len(self.ratings_df) / (self.n_users * self.n_items)) * 100:.2f}%")

# Usage
recommender = RecommenderSystem()
recommender.evaluate_model()
recommender.train()

user_embeddings, movie_embeddings = recommender.get_embeddings()
print(f"User Embeddings Shape: {user_embeddings.shape}")
print(f"Movie Embeddings Shape: {movie_embeddings.shape}")

full_matrix = recommender.get_full_matrix()
print(f"Full Matrix Shape: {full_matrix.shape}")


Total Users: 610
Total Movies: 9742
Matrix Size: 5942620
Total Ratings Available: 100836
% of Filled Matrix: 1.70%
Training started...


Epochs:   0%|          | 1/200 [00:01<03:38,  1.10s/it]

Epoch 1/200, Loss: 12.3617


Epochs:   1%|          | 2/200 [00:02<03:43,  1.13s/it]

Epoch 2/200, Loss: 8.3247


...

Epoch 198/200, Loss: 0.1194


Epochs: 100%|█████████▉| 199/200 [03:34<00:01,  1.14s/it]

Epoch 199/200, Loss: 0.1202


Epochs: 100%|██████████| 200/200 [03:35<00:00,  1.08s/it]

Epoch 200/200, Loss: 0.1196
Training completed.
User Embeddings Shape: (610, 20)
Movie Embeddings Shape: (9742, 20)
Full Matrix Shape: (610, 9742)
